This notebook contains notes on how random number generation in python works

# Introduction to randomness in Python
### Pseudo Random numbers, time, the Mersenne twister and random.seed
Random numbers in general are generated using **pseudo random number generators**, where **pseudo** refers to the fact that these generators are deterministic in how they produce the numbers. (Note that there is a formal theoretical CS definition for pseudorandom distributions, though this is not necessary for understanding).

The generator that underlies Python (by default) is the **mersenne twister**. It involves maintaining an array of 624 32-bit integers which constantly gets updated and refreshed according to **twist** and **temper** operations. The end result of the underlying algorithm is a sequence which only repeats after $2^{19937}−1$ numbers.

```random.seed(num)``` (and similar notations with ```seed()```) allows a program to reproduce random number sequence. It initializes the state array of the running script's mersenne twister based on num.
```python
random.seed(42)        # Set the seed
print(random.random()) # Always outputs 0.6394267984578837
print(random.random()) # Always outputs 0.025010755222666936

random.seed(42)        # Reset the seed
print(random.random()) # Output repeats: 0.6394267984578837
```

### From Distributions
**$Unif(0,1)$ continuous distribution**:
Python generates uniform random numbers within $[0,1)$ (using ```random.random()```) by calling on the mersenne twister to get a 32-bit integer then dividing the value by $2^{32}$ (effectively normalizing it to the 0-1 interval). For other ranges $[a,b]$, we can implement the following:
```python
def unif(a,b):
    x = random.random()
    # Use scaling and shifting
    return a + x*(b-a)
```

**Uniform discrete distribution**:
Python generates discrete uniform random numbers within arbitrary ranges $[a,b]$ by essentially using the following:
```python
# random.randint(a,b) is the actual function
def randint(a,b):
    x = random.random()
    # equivalent to a + floor(u*(b-a+1))
    # below implementation takes advantage
    # of int being essentially a floor
    # since it just truncates
    return a + int(u*(b-a+1))
```

**Normal distribution**:
Python generates normally distributed random variables using the ```random.normalvariate(mu,sigma)``` function where mu is the mean and sigma is the standard deviation. Underyling this function is the **box-muller transform** which uses a uniform random number generators in order to create two independent normally distributed random variables.
```python
# Generating a N(0,1) variable
u1 = random.random()
u2 = random.random()
r = math.sqrt(-2.0 * math.log(u1))
theta = 2.0 * math.pi * u2
z0 = r * math.cos(theta)
# z1 = r * math.sin(theta)

# Scaling and shifting to fit mu and sigma
ans = mu + sigma * z0
```
To derive the box-muller transform, consider the joint pdf of two $N(0,1)$ random variables in the $(x,y)$-coordinate system then convert into the polar coordinate system $(r,theta)$.

**Inverse CDF technique**:
For distributions whose cumulative distribution functions have an analytic inverse $F^{-1}(x)$ (where $F:[0,1]\rightarrow \text{distribution range}$), generating a sequence of random samples from said distributions is simply a matter of coding the inverse CDF formula $F^{-1}(x)$ then plugging in a uniform random variable from ```random.random()```.

**Rejection sampling**:
Rejection sampling is used when you can't invert the CDF easily, but for which you know the pdf function $f(x)$. The idea is to use a simpler distribution which you can sample from and satisfies:
\begin{equation*}
\forall x, f(x) \leq Mg(x)
\end{equation*}
where $f$ and $g$ are pdfs, $f$ being the target and $g$ being the one we are sampling from, and $M$ is a constant known as the envelope constant.

The procedure is to then sample $x$ from the simpler distribution, and $u$ from Unif(0,1), and only accept $x$ if $u\leq\frac{f(x)}{Mg(x)}$ which occurs with probability $\frac{f(x)}{Mg(x)}$. 

```python
# Basic example
# Lets say rand10() generates ints from 1 to 10 inclusive
# You want to generate ints from 1 to 3
# Then you can use the following
def rand3():
    x = rand10()
    while x == 10:
        x = rand10()
        # 1 = 1,4,7 
        # 2 = 2,5,8
        # 3 = 3,6,9
    return x%3+1
```

The caveat with rejection sampling is that it can be slow. 
- For starters, there is practically no upper bound on the time it would take to get an accepted sample since you could just repeatedly draw large $u$ values that get rejected. 
- The general probability of accepting is given by $\frac{1}{M}$ (geometric), so the expected number of iterations before acceptance is given by $M$. Thus choosing the right $g(x)$ such that $M$ is small is important for better runtimes.

### From sequences
**Random permutations/shuffling**:
Suppose we are given an array of length $n$ and we want to create a function to generate a random permutation of it such that the output has a uniform distribution over all possible permutations. Then what we want is to find a way to make an algorithm that chooses one element at a time to fill in positions in an n-array as one would if when generating a random permutation by hand. The **fisher-yates shuffle** is an algorithm that does exactly this but in linear time ($O(n)$). When it comes to shuffling an array in place, the same fisher-yates algorithm is still used.

```python
# Generating a random permutation of n total items
# where the distribution over all possible 
# permutations is uniform
def generate_0(n):
    # Worst case time complexity : O(n^2)
    arr = list(range(n))
    perm = []
    while arr:
        i = random.randint(0,len(arr)-1)
        perm.append(arr.pop(i))
    return perm

def generate_1(n):
    # Upper bound on time complexity: infinite
    # Expected time complexity: nlogn
    arr = []
    while len(arr) < n:
        num = random.randint(1,n)
        if num not in arr:
            arr.append(num)
    return arr

def generate_2(n):
    # Fisher yates shuffle

    # Instead of constructing a list
    # we modify an array in place
    # and we "set" values by going from the end and choosing one of the indexes to swap with before
    # locking in the value.

    # Time complexity: n
    arr = list(range(n))
    for i in range(n-1, 0, -1):
        j = random.randint(0, i)
        arr[i], arr[j] = arr[j], arr[i]
    return arr

def generate(r,n)
    # Truncated fisher yates shuffle but where we choose a 
    # random subset of r elements from the range 0 to n-1 to shuffle
    arr = list(range(n))
    for i in range(n-1, n-r-1, -1):
        j = random.randint(0, i)
        arr[i], arr[j] = arr[j], arr[i]
    return arr[-r:]
```

**Random choices (i.e. n choose r)**:
Suppose we are given an array of length $n$ and want to choose a combination (no regard for order) of $r$ items using a function. This function must generate each combination with equal probability.

```python
def generate(r,n):
    # Brute force
    chosen = set()
    while len(chosen) < r:
        chosen.add(random.randint(0, n-1))
    return tuple(sorted(list(chosen)))

def generate(r,n):
    # Truncated fisher yates shuffle
    arr = list(range(n))
    for i in range(n-1, n-r-1, -1):
        j = random.randint(0, i)
        arr[i], arr[j] = arr[j], arr[i]
    return tuple(sorted(arr[-r:]))
```

Suppose we assign weights to each of the items in the n-array to show the relative probabilities of choosing each element.
```python
# Choosing r elements from the n-array via Efraimidis–Spirakis algorithm
def generate(r,n,weights):
    items = list(range(n))
    keys = []
    for item, w in zip(items, weights):
        # Key = -log(U)/w, U ~ Uniform(0,1)
        u = random.random()
        key = -math.log(u) / w # key is distributed ~ Exp(w)
        keys.append((key, item))
        # By the exponential weights, you can prove that P(X_i is the first) = w_i/(total w)
        # which is exactly what you want.

    # Step 2: Select top-r items by key
    keys.sort()  # sort ascending
    selected = [item for key, item in keys[:r]]
    
    return selected
```
**Random derangements**:
A derangement is a permutation with one cycle, i.e. everything is out of place (nothing is in its original index). To generate derangements, we use sattolo's algorithm, which is simply the fisher-yates shuffle but we make sure not to choose the current element in the swap. This is more clearly understood by reading the code below.

```python
    # Sattolo's algorithm to generate a derangement (only one cycle). 
    def generate(n):
        arr = list(range(n))
        for i in range(n-1, 0, -1):
            j = random.randint(0, i-1)  # main difference with fisher yates: j<i so the current element has to be different from the index.
            arr[i], arr[j] = arr[j], arr[i]
        return arr
```

### Common incorrect algorithms/mistakes and their explanations
Fisher yates shuffle but forwards: This leads to a non-uniform distribution over the possible permutations. The easiest way to see this is to consider the total number of shuffle sequences that are possible (it will not be divisible by the total number of permutations of an n-array for all n, so it cannot be uniformly distributed by the pigeonhole principle).
```python
def generate(r,n)
    arr = list(range(n))
    for i in range(n):
        j = random.randint(0, i)
        arr[i], arr[j] = arr[j], arr[i]
    return arr
```

# On reservoir sampling
**Reservoir sampling** is the process of doing $n$ choose $k$ items, but we have our $n$ as an unknown. In other words we recieve the data array $[x_1,x_2,...]$ element by element and we need an algorithm to choose $k$ items such that every possible combination has equal probability of occuring.

```python
# Single element (k=1)
def generate(arr):
    # In the end every element has probability 1/N of being chosen
    # to derive the above result, consider the probability of arriving and of being replaced.
    reservoir = arr[0]
    # The below uses a for loop with the len(arr) but we can also use a python generator
    for i in range(1,len(arr)):
        x = arr[i]
        u = random.random()
        # probability 1/i of switching out the current element
        if u < 1/i:
            reservoir.pop()
            reservoir.append(x)
    return reservoir

# k elements
def reservoir_sample(stream, k):
    # Overall probability that any element appears in the array is k/N
    # to derive the above result, consider the probability of arriving and of being replaced.
    reservoir = []
    # Fill the reservoir with first k items
    for i, item in enumerate(stream):
        if i < k:
            reservoir.append(item)
        else:
            # Random integer between 0 and i
            j = random.randint(0, i)
            if j < k:
                reservoir[j] = item
    return reservoir
```


# Basic intro to testing and validating "random" generating functions
### Rough/Visual approach: Histogram, QQ-plots and scatter plots
Testing accuracy:
- Histogram: A histogram lets you see whether the distribution of the output of your random function matches $f(x)$.
- QQ-plot: A qq-plot stands for quantile-quantile and plots the empirical quantiles of a bunch of samples from your random function to $f(x)$. This works for functions which are meant to generate values on some interval rather than say a random permutation of an array. 

Testing randomness:
- Scatter plot: A scatter plot of the samples against a lagged version of the samples (e.g. the samples at index $i$ against index $i-1$), can let you observe any autocorrelation between consecutive samples. There should be no obvious linear relationship in the scatterplot since samples are meant to be iid of one another.

### Rigorous approach: Hypothesis testing
General approach for hypothesis testing: Lets say your random number generator is meant to draw from distribution $f(x)$. To construct a hypothesis test on whether your generator actually draws from $f(x)$, we choose a test statistic, e.g. like the sample mean, variance, etc, then calculate the p-value under the null hypothesis that we do have the same distribution and draw conclusions accordingly.

Common methods: 
- The easiest way is to rely on the central limit theorem (of course enforcing that we know the underlying has finite variance and expectation), to perform z and t-tests.
    - z-test: If the value can be assumed to be normally distributed and with known variance. We calculate the p-value based on the probability that they lie in certain sections of the normal distribution.
    - t-test: If the value can be assumed to be normally distributed but with an unknown variance. We calculate the p-value based on the probability that they lie in certain sections of the student t distribution.
    - chi-square test: Mainly concerned with testing variance. Also relies on the central limit theorem.
- A more robust way, especially if we are dealing with smaller number of samples (possibly because it takes a long time for the algorithm to generate a single sample) or a test statistic whose distribution converges too slowly to the normal distribution, would be non parametric approaches that don't assume anything about the underlying
    - Bootstrapping: Generate many resampled datasets (with replacement) from the observed data, compute the test statistic for each, and use this empirical distribution to compute the p-value.
    - Permutation tests: Randomly shuffle or resample labels/signs under the null hypothesis, recompute the statistic for each shuffle, and use the resulting distribution to compute the p-value. It often requires some underlying assumptions about the distribution's shape, for instance if the test was for a mean then the distribution should be symmetric about the mean.

Example of looking at the mean: 
Suppose we have a random generating algorithm ```algo(lambda)``` which should output an exponentially distributed number with rate parameter lambda. Then we can let our null hypothesis be that the mean should be $1/lambda$ if it is distributed exponentially under rate parameter lambda (based on the law of large numbers). By the central limit theorem, the mean will be approximately normal distributed for a large enough number of samples. This assumption can be used to perform z and t-tests as in the below code:

```python
lambda_val = 1
compare = 1/lambda_val # H0: mu = compare
# Two sided one sample z-test
def generate_p_val():
    values = [algo(lambda_val) for i in range(1000)]
    mu = sum(values)/len(values) # Sample mean
    
    assumed_sigma = (1/lambda_val)/np.sqrt(1000) # Assumed variance from H0

    z_statistic = (mu - compare) / assumed_sigma # Scales according to standard normal

    # probability that we get mu if the mean is distributed by N(1/lambda,(1/lambda)^2/1000)
    p_val = 2*(1-scipy.stats.norm.cdf(abs(z_statistic))) # the farther away mu is from compare, the smaller the p_val
    return p_val

# Two sided one sample t-test
def generate_p_val2():
    values = [algo(lambda_val) for i in range(1000)]
    mu = sum(values)/len(values) # Sample mean
    sigma = np.std(values) # Sample variance

    t_statistic = (mu-compare) / (sigma/np.sqrt(1000))
    p_val = 2*(1-scipy.stats.t.cdf(abs(t_statistic), df = 999)) # degrees of freedom = 999
    return p_val

# Bootstrap confidence interval
def generate_p_val3():
    values = [algo(lambda_val) for i in range(1000)]
    mu = np.mean(values)
    
    # Bootstrap resampling
    boot_means = []
    for _ in range(1000):
        resample = np.random.choice(values, size=len(values), replace=True)
        boot_means.append(np.mean(resample))
    
    boot_means = np.array(boot_means)
    
    # Compute two-sided p-value
    # Probability of getting a mean at least as extreme as mu under H0
    diff = abs(mu - compare)
    p_val = np.mean(abs(boot_means - compare) >= diff)
    return p_val

# Permutation test
def generate_p_val4():
    centered = np.array(values) - compare
    T_obs = np.mean(centered)  # observed test statistic
    
    perm_means = []
    for _ in range(num_perm):
        signs = np.random.choice([-1, 1], size=len(centered))
        perm_means.append(np.mean(centered * signs))
    
    perm_means = np.array(perm_means)
    
    # Two-sided p-value
    p_val = np.mean(np.abs(perm_means) >= abs(T_obs)) # If it is the mean should lead to approximately 0.5
    return p_val
```

Example of autocorrelations: 
Suppose now that we want to make sure the random generator ```algo(lambda)``` doesn't create autocorrelated outputs. We can perform the below statistical tests:
- Ljung box (looks at autocorrelation up to lag m)
- Durbin watson (looks at lag 1 autocorrelation)
- Runs test (non parametric)

### Sidenote on rate of convergence to a normal distribution:
The central limit theorem states that the mean of samples taken from a finite variance and mean distirbution converges in distribution to the normal distribution $N(\bar{X},\sigma^2/n)$.

The **berry-esseen theorem** gives an upper bound on how far the sampling distribution of the mean is from the normal:
\begin{equation*}
\sup_x{|F_n(x)-\Phi(x)|}\leq\frac{0.4748*skew}{\sigma^3\sqrt{n}}
\end{equation*}
where $F_n(x)$ is the cdf of the standardized sample mean of n samples,  $\sigma$ is the standard deviation of the underlying distribution and $skew$ is the skew of X (the distribution each sample is assumed to follow).

Using the bound above, we can determine the value of $n$ which will make the error more or less negligible.

# Introduction to Monte carlo
### Basic Monte Carlo procedure
**Monte carlo methods** refer to any method that estimates values by random sampling, e.g. estimating means, partial derivatives, integrals, etc. Suppose we have the known target distribution $p(x)$ whose expected value we want to calculate. Then we can **directly** sample from $p(x)$ repeatedly to get $n$ samples and calculate the sample mean as our estimate for the expected value. A common extension of this is the idea that you want to generate some sort of stochastic process $X_t$ up till time $T$ for which you can easily sample the distribution of the incremental changes $dX$.

### MCMC procedure
Note that below, all references to distribution $p(x)$ means that $p(x)$ is the pdf of our target distribution.

**Markov chain monte carlo** is a group of monte carlo methods which are used when sampling directly from the known target distribution $p(x)$ is difficult to do. The general idea is to create a markov chain $X_0,...,X_n$ with stationary distribution $p(x)$:
- $X_{t+1}$ depends on $X_t$ (hence a markov chain) and the sequence is generated according to bayes rule on prior and posterior distributions.
- After some time $B$ called the **burn-in period**, the samples $X_i$ approximately $\sim p(x)$. 
- We continue simulating the markov chain past the burn-in period and take the samples after that. 
- To reduce autocorrelation between samples from the MCMC, we can pick every k-th value after the burn in period. This is called **thinning**

**Metropolis Hastings**: In the metropolis hastings algorithm, we have a known target $p(x)$ that is hard to sample from and we choose some proposal distribution $q(x|y)$. The proposal distribution is typically something that is 
- easy to sample from/generate 
- has the same support as $p(x)$ (i.e. outputs the same range of values)
- and has the appropriate variance (tradeoff: high variance leads to more rejections and low variance leads to slow exploration and high autocorrelation).

Algorithm steps:
- initialization: start with $x_0$
- proposal step: get $x^*$ from distribution $q(x*|x_t)$ at every time $t+1$.
- acceptance step: accept $x*$ (i.e. update $x_t+1 = x^*$) with probability $\alpha=\min(1,\frac{p(x*)q(x_t|x*)}{p(x_t)q(x*|x_t)})$. This formula can be derived by thinking about the balance between transition probabilities and $p(x)$.


```python
# Known pdf of the target distribution
def p(x):
    return np.exp(-x**4 + x**2)

# we assume x* ~ N(x_t,sigma^2) where sigma is given. This is called the random walk proposal distribution and has the advantage of alpha reducing to just p(x_star)/p(x).
def metropolis_hastings(p, x0=0.0, n_iter=10000, proposal_std=1.0):
    samples = []
    x = x0
    for _ in range(n_iter):
        # propose new sample
        x_star = np.random.normal(x, proposal_std)
        # acceptance ratio
        alpha = min(1, p(x_star)/p(x))
        # accept or reject
        if np.random.rand() < alpha:
            x = x_star
        samples.append(x)
    return np.array(samples) # no burn-in or thinning considered here, just the basic form.
```

**Gibbs sampling**: Suppose that we have a multivariate random variable which we want to simulate. The gibbs sampling procedure essentially does the same thing as metropolis hastings except we have the analytic formula for the conditional pdf for every variable given the others and always accept the proposal.

```python
# Example gibbs for bivariate normal with correlation rho
def gibbs_sampling(rho=0.9, n_iter=10000):
    samples = []
    x, y = 0.0, 0.0  # start at origin

    for _ in range(n_iter):
        # sample x given y
        mean_x = rho * y
        std_x = np.sqrt(1 - rho**2)
        x = np.random.normal(mean_x, std_x)

        # sample y given x
        mean_y = rho * x
        std_y = np.sqrt(1 - rho**2)
        y = np.random.normal(mean_y, std_y)

        # Skip acceptance probability step
        samples.append([x, y])

    return np.array(samples)
```

**Hamiltonian Monte Carlo**: This is an "extension" of sorts of the regular metropolis hastings algorithm. Recall that for MH, we need to choose the right variance of our proposal distribution, and that in general we can have very slow convergence. HMC essentially fixes this by making big steps when we are still far from the target. Note that the notation below considers that we may have a multivariate distribution, so matrices and transposes are involved.

Suppose our target pdf is given by $p(x)$. Then if we define $U(x)=-\log p(x)$, then $p(x)=\frac{1}{Z}e^{-U(x)}$ where Z is some normalizing constant. We start with some initial "position" and "momentum" $(x_t,p_t)$ then propose a new state $(x^*,p^*)$ according to hamiltonian dynamics:
\begin{array}{rl}
    p &\leftarrow p-\epsilon/2 \nabla U(x) \\\\
    x &\leftarrow x+\epsilon M^{-1} p \\\\
    p &\leftarrow p-\epsilon/2 \nabla U(x)
\end{array}
where $\epsilon$ is the step size (a constant) for our dynamics and $\nabla U(x)$ is the gradient of $U$.

Afterwards, we calculate the acceptance probability:
\begin{array}{rl}
H(x,p) &= U(x)+\frac{1}{2}p^TM^{-1}p \\\\
\alpha &= \min (1,e^{-H(x^*,p^*)+H(x_t,p_t)}) 
\end{array}

```python
# Example HMC for a normal distribution.
def U(x):
    return 0.5 * x**2

def grad_U(x):
    return x

def hmc(U, grad_U, x0, n_samples=5000, L=20, epsilon=0.1):
    samples = []
    x = x0

    for _ in range(n_samples):
        p = np.random.normal(0, 1)

        # (x_t,p_t)
        x_current = x
        p_current = p

        # Generate proposal state
        p -= 0.5 * epsilon * grad_U(x)
        for _ in range(L):
            x += epsilon * p
            if _ != L-1:
                p -= epsilon * grad_U(x)
        p -= 0.5 * epsilon * grad_U(x)

        # Accept-reject
        if np.random.rand() < np.exp((U(x_current) + 0.5 * p_current**2) - (U(x) + 0.5 * p**2)):
            samples.append(x)
        else:
            x = x_current
            samples.append(x)

    return np.array(samples)
```

### Quasi Monte Carlo
**Quasi monte carlo** methods are just like monte carlo methods in that we use sampling of many points to approximate values, particularly the values of integrals. However, one main difference is that quasi monte carlo methods use entirely deterministic sequences which are more evenly distributed across the space that we want to integrate over.

The **sobol sequence** is one of the most commonly used sequences. It fills up the d-dimensional cube space $[0,1]^d$ uniformly such that there is no clustering of points as the sequence is generated. The error of an integral calculated using the sobol sequence QMC method is bounded by $O((logN)^d/N)$ where N is the number of points sampled.

```python
# Example
# Function to integrate
def f(x):
    return np.exp(-x**2)

# Number of points
N = 1000

sampler = qmc.Sobol(d=1, scramble=True)
x_qmc = sampler.random(N)
I_qmc = np.mean(f(x_qmc))

x_mc = np.random.rand(N, 1)
I_mc = np.mean(f(x_mc))

print(f"QMC estimate: {I_qmc}")
print(f"MC estimate: {I_mc}")
```

Sidenote: There are many other sequences you can explore, but I am not sure how useful anything beyond the sobol sequence is.